# 用 SFTTrainer 实现有监督微调

本教程将展示如何使用 `trl` 代码库中的 `SFTTrainer` 来微调 `HuggingFaceTB/SmolLM2-135M`。文中代码模块将会展示具体步骤。你可以根据难度选取在不同数据集上微调。

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>练习：用 SFTTrainer 微调 SmolLM2</h2>
    <p>从 HuggingFace 上选取一个数据集然后微调模型 </p> 
    <p><b>难度等级</b></p>
    <p>🐢 使用 `HuggingFaceTB/smoltalk` 数据集</p>
    <p>🐕 尝试 `bigcode/the-stack-smol` 数据集，在 `data/python` 这个子集上微调一个代码生成模型</p>
    <p>🦁 选取一个你感兴趣的领域，使用真实世界数据集，为你的特定应用微调模型</p>
</div>

In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# 用基准模型生成

我们首先看看不使用聊天模板的基础模型是什么效果。

In [ ]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## 准备数据集

接下来我们载入一个简单数据集，并转换格式。这个数据集需要结构化为“输入-输出”这样成对的形式，其中输入是一个提示语，输出就是我们期待的模型回答。

**TRL 会自动根据聊天模板转换输入信息。**这些信息需要写成一个字典（dict）的列表，其中 key 是 `role` 和 `content`，分别代表哪个角色以及对应的话语。

In [ ]:
# Load a sample dataset
from datasets import load_dataset

# TODO: 稍后你可以使用你自己的数据集，替换以下的路径、名字等参数配置
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 如果你的数据集不是 TRL 能转换成聊天模板的格式，你还需要额外处理一下。参考[聊天模板](../chat_templates_cn.md)

## 配置 SFTTrainer

我们还需要配置 `SFTTrainer` 的各项参数，来控制训练过程。这包括训练步数、batch size、学习率，以及评估策略。你需要根据你自己的情况和计算资源来调节。

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 根据你自己的数据集调节 SFTTrainer 的参数。假如你用的是 `bigcode/the-stack-smol` 数据集，那你就需要选择 `content` 这一列。

## 训练模型

 当上述配置完成后，我们就可以训练了。这个过程将遍历数据集、计算损失函数、更新模型参数等。

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>额外练习：用微调过的模型来生成</h2>
    <p>🐕 使用微调过的模型来生成回答，就像之前用基准模型一样。对比一下效果</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: 用微调过的模型生成相同的回答，看看有没有变化改进

## 💐 完成了！

本文提供了使用 `SFTTrainer` 微调 `HuggingFaceTB/SmolLM2-135M` 的指南。依照这些步骤，你可以将模型适配到各种特殊任务场景上。如果你觉得这个课程还不错，你还可以尝试这些：

- 使用这个 notebook 完成难点分级更高的任务
- 在 GitHub 上评审别人的 pull request
- 通过提出 Issue 或 PR 进一步改进我们的课程资料